## TYTANチュートリアル（カクラス）

2023年5月5日

ビネクラ安田

出典：[量子アニーリング（D-wave）でカクラス（Kakurasu）を解く](https://vigne-cla.com/21-17/)


### 問題
カクラス（Kakurasu）は掛け算の計算が必要な <s>面倒な</s> 変わったパズル。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-17_1.png" width = 20%>
</div>

答えはこちら。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-17_2.png" width = 20%>
</div>

上辺・左辺の小さな数字が各列のマスの重みを表す。白マスを０、黒マスを１と考えて、各マスの重みをかけて合計した数値が右辺・下辺の数字と一致するように塗る。例えば、一番上の行は １×１＋２×０＋３×０＋４×１＝５ を意味している（赤字がマスの色に対応）。

### QUBOモデルでは何が設定できるか？（おさらい）

**<font color="red">「n個の量子ビットからm個を1にする」</font>**

例）3個の量子ビットから2個を1にする
```
H = (q1 + q2 + q3 - 2)**2
```

今回はこの基本の条件設定を改造して使う。その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### 条件設定

16個の量子ビットを各マスに対応させる。結果が１なら黒マス（＝重み有効）と考える。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-17_3-300x293.png" width = 20%>
</div>

一行だけ取り出して考える。この行は「1×q0＋2×q1＋3×q2＋4×q3＝5」にしたい。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-17_4-768x120.png" width = 55%>
</div>

ここで基本の条件設定を思い浮かべる。「４個の量子ビットからｍ個を１にする」は次の式。

```
H = (q1 + q2 + q3 - 2)**2
```

これはつまり、**４個の量子ビットを足した値がｍになる**ということ。合計値がｍであればエネルギーは最小で、合計値がｍからずれるとエネルギーが高くなる。

ここで、４つの量子ビットを足す際に重みをかける。

```
H = (1*q0 + 2*q1 + 3*q2 + 4*q3 - 5)**2
```

実はこれで式が完成。**「1×q0＋2×q1＋3×q2＋4×q3」の部分が５のときにエネルギーは最小**となり、５からずれるとエネルギーが高くなる。少し柔軟な発想が必要だ。

## コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [2]:
from sympy import Symbol
from tytan import qubo, sampler
import numpy as np

#量子ビットを用意する
q00 = Symbol('q00')
q01 = Symbol('q01')
q02 = Symbol('q02')
q03 = Symbol('q03')
q04 = Symbol('q04')
q05 = Symbol('q05')
q06 = Symbol('q06')
q07 = Symbol('q07')
q08 = Symbol('q08')
q09 = Symbol('q09')
q10 = Symbol('q10')
q11 = Symbol('q11')
q12 = Symbol('q12')
q13 = Symbol('q13')
q14 = Symbol('q14')
q15 = Symbol('q15')

#各行、「重み付き合計値が指定の値になる」
H = 0
H += (1*q00 + 2*q01 + 3*q02 + 4*q03 - 5)**2
H += (1*q04 + 2*q05 + 3*q06 + 4*q07 - 7)**2
H += (1*q08 + 2*q09 + 3*q10 + 4*q11 - 5)**2
H += (1*q12 + 2*q13 + 3*q14 + 4*q15 - 8)**2

#各列、「重み付き合計値が指定の値になる」
H += (1*q00 + 2*q04 + 3*q08 + 4*q12 - 5)**2
H += (1*q01 + 2*q05 + 3*q09 + 4*q13 - 3)**2
H += (1*q02 + 2*q06 + 3*q10 + 4*q14 - 9)**2
H += (1*q03 + 2*q07 + 3*q11 + 4*q15 - 7)**2


#コンパイル
QUBO, offset = qubo.Compile(H).get_qubo()

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(QUBO, shots=500)

#上位5件
for r in result[:5]:
    print(r)

#上位2件
print(np.array(list(result[0][0].values()), int).reshape(4, 4))
print(np.array(list(result[1][0].values()), int).reshape(4, 4))

[{'q00': 1.0, 'q01': 0.0, 'q02': 0.0, 'q03': 1.0, 'q04': 0.0, 'q05': 0.0, 'q06': 1.0, 'q07': 1.0, 'q08': 0.0, 'q09': 1.0, 'q10': 1.0, 'q11': 0.0, 'q12': 1.0, 'q13': 0.0, 'q14': 1.0, 'q15': 1.0}, -327.0, 236]
[{'q00': 0.0, 'q01': 0.0, 'q02': 0.0, 'q03': 1.0, 'q04': 0.0, 'q05': 0.0, 'q06': 1.0, 'q07': 1.0, 'q08': 0.0, 'q09': 1.0, 'q10': 1.0, 'q11': 0.0, 'q12': 1.0, 'q13': 0.0, 'q14': 1.0, 'q15': 1.0}, -325.0, 73]
[{'q00': 1.0, 'q01': 0.0, 'q02': 0.0, 'q03': 1.0, 'q04': 0.0, 'q05': 0.0, 'q06': 1.0, 'q07': 1.0, 'q08': 1.0, 'q09': 1.0, 'q10': 1.0, 'q11': 0.0, 'q12': 0.0, 'q13': 0.0, 'q14': 1.0, 'q15': 1.0}, -324.0, 41]
[{'q00': 1.0, 'q01': 0.0, 'q02': 1.0, 'q03': 0.0, 'q04': 0.0, 'q05': 0.0, 'q06': 1.0, 'q07': 1.0, 'q08': 0.0, 'q09': 1.0, 'q10': 1.0, 'q11': 0.0, 'q12': 1.0, 'q13': 0.0, 'q14': 1.0, 'q15': 1.0}, -324.0, 7]
[{'q00': 0.0, 'q01': 1.0, 'q02': 1.0, 'q03': 0.0, 'q04': 1.0, 'q05': 1.0, 'q06': 0.0, 'q07': 1.0, 'q08': 1.0, 'q09': 0.0, 'q10': 1.0, 'q11': 0.0, 'q12': 0.0, 'q13': 0.0, 'q

一番目の解は模範解答の通りである。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-17_2.png" width = 20%>
</div>